<a href="https://colab.research.google.com/github/J-Yuna/myfirst/blob/main/SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import svm




In [15]:
iris = datasets.load_iris()
x = df[df.columns[:-1]]
y=df['target']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

NameError: ignored